In [ ]:
import brightway2 as bw
bw.projects.set_current('wurst')
list(bw.databases)

In [ ]:
ECOINVENT_DB_NAME = 'ecoinvent3.7.1_wurst' # heat and electricity replacement processes were added beforehand to ecoinvent 3.7.1

assert ECOINVENT_DB_NAME in bw.databases
assert len(bw.Database(ECOINVENT_DB_NAME))

In [ ]:
import wurst as w
ecoinvent = w.extract_brightway2_databases([ECOINVENT_DB_NAME])

In [ ]:
# Define scenario
scenario = '2050' # predefined: 2030, 2050, 'Run-of-river'

# Change hydrogen supply from fossil to electrolysis
hydrogen_fossil_to_el = True # choice only for 2050 and Run-of-river

In [ ]:
# Search for new activity that will to be used to replace heat processes in ecoinvent

individual_heat_process = False # if you insert your own heat process, otherwise electricity will be used as heat
if scenario == '2050' or scenario == 'Run-of-river':
    if individual_heat_process == True:
        name = 'heat production, in chemical industry, future' # replacement process has to be created and copied into the database
        wfilter = [w.equals("name", name)]
        heat_amount = 1
    else:
        name = 'market for electricity, medium voltage'
        location = 'DE'
        filter_1 = [w.equals("name", name)]
        filter_2 = [w.equals("location", location)]
        wfilter = filter_1 + filter_2
        heat_amount = 1/3.6 # assuming electiricty to heat, neglecting losses, unit conversion
        
    best_match = w.get_one(ecoinvent, *wfilter)
    print('NAME')
    print(best_match['name'])
    print('REFERENCE PRODUCT')
    print((best_match['reference product']))
    print('LOCATION')
    print(best_match['location'])
    print('EXCHANGES SUMMARIZED')
    print(best_match['exchanges'])

    for idx, item in enumerate(best_match['exchanges']):
        print(idx)
        print(item)

    print('')
    print('EXCHANGES')
    print(best_match['exchanges'])
    print('')
    print('REFERENCE PRODUCT')
    print(best_match['reference product'])


In [ ]:
# Exchange heat activities

if scenario == '2050' or scenario == 'Run-of-river':
    print('STARTING EXCHANGING HEAT INPUTS')
    matches = w.get_many(ecoinvent, w.either(w.contains('reference product', 'heat, air-water heat pump 10kW'),
                                             w.contains('reference product', 'heat, borehole heat pump'),
                                             w.contains('reference product', 'heat, central or small-scale, biomethane'),
                                             w.contains('reference product', 'heat, central or small-scale, Jakobsberg'),
                                             w.contains('reference product', 'heat, central or small-scale, natural gas'),
                                             w.contains('reference product', 'heat, central or small-scale, natural gas and heat pump, Jakobsberg'),
                                             w.contains('reference product', 'heat, central or small-scale, natural gas, Jakobsberg'),
                                             w.contains('reference product', 'heat, central or small-scale, other than natural gas'),
                                             w.contains('reference product', 'heat, diffusion absorption heat pump'),
                                             w.contains('reference product', 'heat, district or industrial, natural gas'),
                                             w.contains('reference product', 'heat, district or industrial, other than natural gas'),
                                             #w.contains('reference product', 'heat, for reuse in municipal waste incineration only'),
                                             #w.contains('reference product', 'heat, from steam, in chemical industry'), #predecessors already contained
                                             w.contains('reference product', 'heat, future'),
                                             w.contains('reference product', 'heat, solar+electric, multiple-dwelling, for hot water'),
                                             w.contains('reference product', 'heat, solar+gas, multiple-dwelling, for hot water'),
                                             w.contains('reference product', 'heat, solar+gas, one-family house, for combined system'),
                                             w.contains('reference product', 'heat, solar+gas, one-family house, for hot water'),
                                             w.contains('reference product', 'heat, solar+wood, one-family house, for combined system'),
                                            ))
    print(matches)

    for ds in matches:        
        print('EXCHANGES BEFORE OPERATION')
        print(ds['exchanges'])
        if ds['name'] != name:
            for idx, item in enumerate(ds['exchanges']): 
                print('BEFORE')
                print(idx)
                print(item)
                item['amount'] = 2.2250738585072014e-308 # 0 did not work in the database linking, snmallest number available used
                if item['type']  == 'production': 
                    item['amount'] = 1

        ds['exchanges'].append({
        'amount': heat_amount,
        'input': (best_match['database'], best_match['code']),
        'type': 'technosphere',
        'name': best_match['name'],
        'location': best_match['location']
        })

        for idx, item in enumerate(ds['exchanges']):    
            print('EXCHANGES AFTER OPERATION')
            print(idx)
            print(item)

In [ ]:
# Exchange electricity activities
# Move low and medium to high voltage

from wurst.ecoinvent.electricity_markets import (
    empty_low_voltage_markets,
    empty_medium_voltage_markets,
    empty_high_voltage_markets,
    move_all_generation_to_high_voltage,
)

move_all_generation_to_high_voltage (ecoinvent)

In [ ]:
# Load excel with new amounts for high voltage eletricity

from openpyxl import load_workbook
wb = load_workbook(filename="data/new_electricity_mix.xlsx")
ws = wb[scenario]
PRODUCERS = [(row, ws.cell(row=row, column=4).value) for row in range(2, 41)]
data = {tech:ws.cell(row=row_index, column=1).value for row_index, tech in PRODUCERS
       }

print(data)

In [ ]:
# Find market which values will be replaced by new amounts of excel

name = 'market for electricity, high voltage'
location = 'DE'
reference_product = 'electricity, high voltage'
filter_1 = [w.equals("name", name)]
filter_2 = [w.equals("location", location)]
filter_3 = [w.equals("reference product", reference_product)]
wfilter = filter_1 + filter_2 + filter_3

best_match = w.get_one(ecoinvent, *wfilter)
print('NAME')
print(best_match['name'])
print('REFERENCE PRODUCT')
print((best_match['reference product']))
print('LOCATION')
print(best_match['location'])
print('EXCHANGES SUMMARIZED')
print(best_match['exchanges'])

for idx, item in enumerate(best_match['exchanges']):
    print(idx)
    print(item)

print('')
print('EXCHANGES')
print(best_match['exchanges'])
print('')
print('REFERENCE PRODUCT')
print(best_match['reference product'])


In [ ]:
# Replace market for electricity, high voltage DE with new scenario values from excel file

for idx, item in enumerate(best_match['exchanges']): 
    if item['name'] in data:
        item['amount'] = data[item['name']]

In [ ]:
# Search for new activity that is going to be used for high voltage electricity processes

name = 'market for electricity, high voltage'
location = 'DE'
reference_product = 'electricity, high voltage'
filter_1 = [w.equals("name", name)]
filter_2 = [w.equals("location", location)]
filter_3 = [w.equals("reference product", reference_product)]
wfilter = filter_1 + filter_2 + filter_3

best_match = w.get_one(ecoinvent, *wfilter)
print('NAME')
print(best_match['name'])
print('REFERENCE PRODUCT')
print((best_match['reference product']))
print('LOCATION')
print(best_match['location'])
print('EXCHANGES SUMMARIZED')
print(best_match['exchanges'])

for idx, item in enumerate(best_match['exchanges']):
    print(idx)
    print(item)

print('')
print('EXCHANGES')
print(best_match['exchanges'])
print('')
print('REFERENCE PRODUCT')
print(best_match['reference product'])

In [ ]:
# Exchange heat activities

print('STARTING EXCHANGING ELECTRICITY INPUTS')
matches = w.get_many(ecoinvent, w.either(w.contains('reference product', 'electricity, high voltage'),
                                         w.contains('reference product', 'electricity, high voltage, aluminium industry'),
                                         w.contains('reference product', 'electricity, high voltage, cobalt industry'),
                                         w.contains('reference product', 'electricity, high voltage, for internal use in coal mining'),
                                         w.contains('reference product', 'electricity, high voltage, for Swiss Federal Railways'),
                                         w.contains('reference product', 'electricity, high voltage, label-certified'),
                                        ))


for ds in matches:        
    print('EXCHANGES BEFORE OPERATION')
    print(ds['exchanges'])
    
    if (ds['name'] != name and ds['location'] != 'DE') or ds['location'] != 'DE': # avoid market for electricity, high voltage as well as 'DE' because this is the activity which will be used
        for idx, item in enumerate(ds['exchanges']): 
            print('BEFORE')
            print(idx)
            print(item)
            item['amount'] = 2.2250738585072014e-308 # 0 did not work
            if item['type']  == 'production': 
                item['amount'] = 1

        ds['exchanges'].append({
        'amount': 1,
        'input': (best_match['database'], best_match['code']),
        'type': 'technosphere',
        'name': best_match['name'],
        'location': best_match['location']
        })
        
        for idx, item in enumerate(ds['exchanges']):    
            print('EXCHANGES AFTER OPERATION')
            print(idx)
            print(item)

In [ ]:
# Search for new activity that is going to be used for hydrogen

if hydrogen_fossil_to_el == True and scenario != '2030':

    name = 'chlor-alkali electrolysis, membrane cell'
    location = 'RER'
    reference_product = 'hydrogen, liquid'
    filter_1 = [w.equals("name", name)]
    filter_2 = [w.equals("location", location)]
    filter_3 = [w.equals("reference product", reference_product)]
    wfilter = filter_1 + filter_2 + filter_3
    
best_match = w.get_one(ecoinvent, *wfilter)
print('NAME')
print(best_match['name'])
print('REFERENCE PRODUCT')
print((best_match['reference product']))
print('LOCATION')
print(best_match['location'])
print('EXCHANGES SUMMARIZED')
print(best_match['exchanges'])

for idx, item in enumerate(best_match['exchanges']):
    print(idx)
    print(item)

print('')
print('EXCHANGES')
print(best_match['exchanges'])
print('')
print('REFERENCE PRODUCT')
print(best_match['reference product'])

In [ ]:
# Exchange hydrogen activities

if hydrogen_fossil_to_el == True and scenario != '2030':
    print('STARTING EXCHANGING HYDROGEN INPUTS')
    matches = w.get_many(ecoinvent, w.either(w.contains('name', 'hydrogen cracking, APME'),
                                            ))
    
    for ds in matches:        
        print('EXCHANGES BEFORE OPERATION')
        print(ds['exchanges'])
        if ds['name'] != name:
            for idx, item in enumerate(ds['exchanges']): 
                print('BEFORE')
                print(idx)
                print(item)
                item['amount'] = 2.2250738585072014e-308 # 0 did not work
                if item['type']  == 'production': 
                    item['amount'] = 1

            ds['exchanges'].append({
            'amount': 1,
            'input': (best_match['database'], best_match['code']),
            'type': 'technosphere',
            'name': best_match['name'],
            'location': best_match['location']
            })

            for idx, item in enumerate(ds['exchanges']):    
                print('EXCHANGES AFTER OPERATION')
                print(idx)
                print(item)

In [ ]:
# Write new database

if hydrogen_fossil_to_el == True and scenario != '2030':
    DB_NAME_ADD = '_H2ely'
else: 
    DB_NAME_ADD = ''
    
NEW_DB_NAME = f"{ECOINVENT_DB_NAME}_{scenario}{DB_NAME_ADD}"
w.write_brightway2_database(ecoinvent, NEW_DB_NAME)

# Check for new databases
list(bw.databases)